In [ ]:
project = %env GOOGLE_PROJECT
bucket = %env WORKSPACE_BUCKET
genomic_location = %env CDR_STORAGE_PATH
mt_wgs_path = %env WGS_HAIL_STORAGE_PATH
CURRENT_DATASET = %env WORKSPACE_CDR
OAUTH_TOKEN=!gcloud auth application-default print-access-token
OAUTH_TOKEN=OAUTH_TOKEN[0]
project, bucket, genomic_location, mt_wgs_path, CURRENT_DATASET

In [ ]:
%%capture
%env HTSLIB_CONFIGURE_OPTIONS "--enable-gs"
%env GCS_OAUTH_TOKEN=$OAUTH_TOKEN
%env GCS_REQUESTER_PAYS_PROJECT=$project

!pip3 install pysam -v --force-reinstall
import pysam

In [ ]:
# # Please download HLAVBSeq for usage
!wget http://nagasakilab.csml.org/hla/HLAVBSeq.jar

In [ ]:
# # Older fetch code for chr6 region in selected sample
# samfile = pysam.AlignmentFile(aou_path + cramfile, "rc")
# #Example for fetching via pysam
# for x in samfile.fetch("chr6", 0, 100000):
#     print(str(x))

# # commands below are generalized and more specifically listed in AoU workbench
wgs_path = "/some/path"
sample = "some_sample.cram"

In [ ]:
"""Major step to load selected file and pre-select region during wkspc import."""
cramfile = pysam.AlignmentFile(wgs_path + sample, "rc")
localcram = pysam.AlignmentFile(sample,"wb",template=cramfile)
for x in cramfile.fetch("chr6"):
    localcram.write(x)
localcram.close()
cramfile.close()
!ls
#Note: 3gb

In [ ]:
"""Subsequent steps are automated in hla_aou_v*.py scripts."""
!samtools view -bo mapped.bam -L /project_path/hg38.bed sample

In [1]:
!samtools sort -n -o sort.bam mapped.bam

[E::hts_open_format] Failed to open file "mapped.bam" : No such file or directory
samtools sort: can't open "mapped.bam": No such file or directory


In [ ]:
!samtools fixmate -O bam sort.bam fixmate.bam

In [ ]:
!samtools fastq -n -0 mapped.0.fastq -s mapped.s.fastq -1 mapped.1.fastq -2 mapped.2.fastq fixmate.bam

In [ ]:
!samtools view -bh -f 12 -o unmapped.bam sample

In [ ]:
!samtools fastq -n -0 unmapped.0.fastq -s unmapped.s.fastq -1 unmapped.1.fastq -2 unmapped.2.fastq unmapped.bam

In [ ]:
#!../../bwa/bwa index alleles.gen.fasta
!../../bwa/bwa mem -t -8 -P -L 10000 -a alleles.gen.fasta mapped.1.fastq mapped.2.fastq > totest.sam

In [ ]:
# HLAVBSeq run
!java -Xmx12G -jar ./HLAVBSeq.jar alleles.gen.fasta totest.sam result.txt --alpha_zero 0.01 --is_paired

In [ ]:
!head result.txt